# Glassdoor 데이터 전처리

- **Type of ownership**: 기업의 소유 형태를 3가지 그룹으로 단순화
- **Rating**: 기업 평점의 이상치를 제거하고 평균값으로 대체

In [2]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv('glassdoor_jobs.csv')

## Type of ownership 전처리

기업의 소유 유형을 다음 기준으로 단순화:
- **Private**: 사기업, 자회사, 계약직, 개인 회사
- **Public**: 상장사
- **Public Service**: 정부, 비영리, 교육, 병원

In [3]:
# ─────────────────────────────────────────────
# Type of ownership 전처리
# ─────────────────────────────────────────────
print("[Type of ownership] 원본 유니크 값 및 개수:")
print(df['Type of ownership'].value_counts(dropna=False), "\n")

# 최빈값 계산 (결측 대체용)
most_common_ownership = df['Type of ownership'].mode()[0]
print(f"최빈값: '{most_common_ownership}'\n")

# 전처리 함수 정의
def classify_ownership(value):
    # - "-1", "Unknown", "Other Organization"은 실제 기업 구조 파악 불가
    # - 단순히 제거하거나 Others로 둘 경우 분석/모델링에서 해석성 및 예측 안정성 저하
    # - 최빈값으로 대체하여 데이터 누락 없이 편입
    if value in ["-1", "Unknown", "Other Organization"]:
        value = most_common_ownership

    # Private 그룹
    # - "Company - Private": 사기업
    # - "Subsidiary or Business Segment": 대부분 상위 사기업 소속
    # - "Contract": 사기업 중심의 고용 형태
    # - "Private Practice / Firm": 소규모 독립 사기업으로 판단
    # 따라서 이들은 "Private"으로 통합
    if value in [
        "Company - Private", "Subsidiary or Business Segment",
        "Contract", "Private Practice / Firm"
    ]:
        return "Private"

    # Public 그룹
    # - "Company - Public": 상장 대기업 또는 주식 공개 기업
    # 독립적인 경제적 규모와 해석을 위해 그룹 유지
    elif value == "Company - Public":
        return "Public"

    # Public Service 그룹
    # - "Nonprofit Organization": 공익 목적
    # - "Government": 공공 부문
    # - "Hospital": 보건 서비스
    # - "College / University", "School / School District": 교육 기관
    # 모두 공공 목적 중심으로, 유사한 고용 특성 고려
    elif value in [
        "Nonprofit Organization", "Government",
        "Hospital", "College / University", "School / School District"
    ]:
        return "Public Service"

# 새로운 피쳐 생성
df['Ownership_Grouped'] = df['Type of ownership'].apply(classify_ownership)

# 결과 출력
print("[Ownership_Grouped] 전처리 결과:")
print(df['Ownership_Grouped'].value_counts(), "\n")

[Type of ownership] 원본 유니크 값 및 개수:
Type of ownership
Company - Private                 532
Company - Public                  237
Nonprofit Organization             65
Subsidiary or Business Segment     40
Government                         17
College / University               15
Hospital                           15
-1                                 11
Unknown                            11
Other Organization                  5
Contract                            5
School / School District            2
Private Practice / Firm             1
Name: count, dtype: int64 

최빈값: 'Company - Private'

[Ownership_Grouped] 전처리 결과:
Ownership_Grouped
Private           605
Public            237
Public Service    114
Name: count, dtype: int64 



## Rating 전처리
- `-1.0`은 평점이 없는 경우 → 결측치로 간주, 평균값 대체

In [4]:
# ─────────────────────────────────────────────
# Rating 전처리
# ─────────────────────────────────────────────
print("[Rating] 유니크 값 및 이상치 확인:")
print(df['Rating'].value_counts(dropna=False), "\n")

# -1.0을 결측치로 간주하고 평균값으로 대체, 분포가 왜곡되지 않았으므로 안전
df['Rating'] = df['Rating'].replace(-1.0, np.nan)
rating_mean = df['Rating'].mean(skipna=True)
df['Rating'] = df['Rating'].fillna(rating_mean)

# 결과 출력
print("[Rating] 전처리 결과:")
print(f"대체된 평균값: {rating_mean:.3f}")
print(f"고유값 개수: {df['Rating'].nunique()}")
print(f"상위 값:\n{df['Rating'].value_counts().head()}")

[Rating] 유니크 값 및 이상치 확인:
Rating
 3.8    75
 3.7    69
 3.9    69
 3.6    56
 4.0    56
 3.5    53
 3.4    47
 4.4    46
 3.3    44
 4.2    41
 4.3    39
 4.7    38
 4.1    37
 3.2    36
-1.0    34
 3.1    30
 5.0    28
 2.9    22
 4.5    19
 4.6    18
 3.0    17
 2.7    17
 4.8    14
 2.6    14
 2.4     8
 2.8     7
 2.1     5
 4.9     4
 2.3     4
 2.2     3
 1.9     3
 2.5     3
Name: count, dtype: int64 

[Rating] 전처리 결과:
대체된 평균값: 3.771
고유값 개수: 32
상위 값:
Rating
3.8    75
3.7    69
3.9    69
3.6    56
4.0    56
Name: count, dtype: int64
